In [1]:
import fast_stats
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

### Compared to scikit-learn

In [2]:
SIZE = (10, 512, 512)

In [3]:
pred = np.random.randint(0, 2, SIZE).astype(bool).flatten()
actual = np.random.randint(0, 2, SIZE).astype(bool).flatten()

In [4]:
%%timeit
_ = precision_score(actual, pred)

584 ms ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
_ = recall_score(actual, pred)

569 ms ± 592 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
_ = f1_score(actual, pred)

572 ms ± 3.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# don't actually need to flatten them for fast-stats
pred = np.random.randint(0, 2, SIZE).astype(bool)
actual = np.random.randint(0, 2, SIZE).astype(bool)

In [8]:
%%timeit
_ = fast_stats.binary_precision(actual, pred)

3.2 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
_ = fast_stats.binary_recall(actual, pred)

3.25 ms ± 89.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
_ = fast_stats.binary_f1_score(actual, pred)

4.23 ms ± 84.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
assert np.allclose(
    fast_stats.binary_precision(actual.flatten(), pred.flatten()),
    precision_score(actual.flatten(), pred.flatten())
)
assert np.allclose(
    fast_stats.binary_recall(actual.flatten(), pred.flatten()),
    recall_score(actual.flatten(), pred.flatten())
)
assert np.allclose(
    fast_stats.binary_f1_score(actual.flatten(), pred.flatten()),
    f1_score(actual.flatten(), pred.flatten())
)

### Compared to numpy

In [18]:
pred, actual = pred.astype(np.uint8), actual.astype(np.uint8)

In [19]:
%%timeit
_ = fast_stats.binary_precision(pred, actual)

2.5 ms ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%%timeit
_ = np.logical_and(actual, pred).sum() / pred.sum()

4.11 ms ± 26.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
